#### Lesson 3: Reflection and Blogpost Writing

#### Setup

In [3]:
#r "nuget:AutoGen"

using AutoGen.Core;
using AutoGen.OpenAI;
using AutoGen.OpenAI.Extension;
using Azure.AI.OpenAI;
using System.Threading;

var openAIKey = Environment.GetEnvironmentVariable("OPENAI_API_KEY") ?? throw new Exception("Please set the OPENAI_API_KEY environment variable.");
var openAIModel = "gpt-3.5-turbo";
var openaiClient = new OpenAIClient(openAIKey);


Installed Packages AutoGen, 0.0.16

The Task!

In [1]:
var task = """
    Write a concise but engaging blogpost about
    DeepLearning.AI. Make sure the blogpost is
    within 100 words.
    """;

Create a writer agent

In [5]:
var writer = new OpenAIChatAgent(
    openAIClient: openaiClient,
    name: "Writer",
    modelName: openAIModel,
    systemMessage: """
    You are a writer. You write engaging and concise blogpost (with title) on given topics.
    You must polish your writing based on the feedback you receive and give a refined version.
    Only return your final work without additional comments."
    """)
    .RegisterMessageConnector()
    .RegisterPrintMessage();

    var reply = await writer.SendAsync(task);

from: Writer
Title: Unleashing the Power of Deep Learning with DeepLearning.AI

Dive into the world of artificial intelligence with DeepLearning.AI! Founded by renowned AI expert Andrew Ng, this platform offers top-notch courses on deep learning, machine learning, and more. Whether you're a beginner or a seasoned professional, DeepLearning.AI provides the tools and knowledge to help you master the latest AI technologies. With hands-on projects and expert-led instruction, you'll be on your way to becoming an AI expert in no time. Join the AI revolution today with DeepLearning.AI!



Adding reflection by creating a critic agent to reflect on the work of the writer agent.

In [6]:
var critic = new OpenAIChatAgent(
    openAIClient: openaiClient,
    name: "Critic",
    modelName: openAIModel,
    systemMessage: """
    You are a critic. You review the work of the writer and provide constructive feedback to help improve the quality of the content."
    """)
    .RegisterMessageConnector()
    .RegisterPrintMessage();

var conversation = await critic.SendAsync(
    receiver: writer,
    message: task,
    maxRound: 3);

// Use the last message in the conversation as the final result
var res = conversation.Last();

from: Writer
Title: "Unlocking the Power of Deep Learning with DeepLearning.AI"

Dive into the world of artificial intelligence with DeepLearning.AI, a platform revolutionizing education in deep learning. Led by industry expert Andrew Ng, this online hub offers top-notch courses, hands-on projects, and a vibrant community of learners. Whether you're a beginner or a seasoned professional, DeepLearning.AI provides the tools and resources to master the complexities of deep learning. With a diverse range of topics and flexible learning options, you can upskill at your own pace. Join the millions of learners worldwide and unleash your potential in AI with DeepLearning.AI.

from: Critic
This blog post is concise and engaging, effectively capturing the essence of DeepLearning.AI and its offerings. The title is attention-grabbing and sets the tone for the content. The content itself provides a good overview of the platform, highlighting key features such as courses, projects, and community sup

#### Nested chat using middleware pattern
Python AutoGen allows you to create a Json object task list and pass it to agent using `register_nested_chats`
This is not supported in AutoGen.Net.
But you can achieve the same using the middleware pattern.

In [7]:
var SEOReviewer = new OpenAIChatAgent(
    openAIClient: openaiClient,
    name: "SEO_Reviewer",
    modelName: openAIModel,
    systemMessage: """
    You are an SEO reviewer, known for your ability to optimize content for search engines, ensuring that it ranks well and attracts organic traffic.
    Make sure your suggestion is concise (within 3 bullet points), concrete and to the point.
    Begin the review by stating your role.
    """)
    .RegisterMessageConnector()
    .RegisterPrintMessage();

In [8]:
var LegalReviewer = new OpenAIChatAgent(
    openAIClient: openaiClient,
    name: "Legal_Reviewer",
    modelName: openAIModel,
    systemMessage: """
    You are a legal reviewer, known for your ability to ensure that content is legally compliant and free from any potential legal issues.
    Make sure your suggestion is concise (within 3 bullet points), concrete and to the point.
    Begin the review by stating your role.
    """)
    .RegisterMessageConnector()
    .RegisterPrintMessage();

In [9]:
var EthicsReviewer = new OpenAIChatAgent(
    openAIClient: openaiClient,
    name: "Ethics_Reviewer",
    modelName: openAIModel,
    systemMessage: """
    You are an ethics reviewer, known for your ability to ensure that content is ethically sound and free from any potential ethical issues.
    Make sure your suggestion is concise (within 3 bullet points), concrete and to the point.
    Begin the review by stating your role.
    """)
    .RegisterMessageConnector()
    .RegisterPrintMessage();

In [10]:
var MetaReviewer = new OpenAIChatAgent(
    openAIClient: openaiClient,
    name: "Meta_Reviewer",
    modelName: openAIModel,
    systemMessage: """
    You are a meta reviewer, you aggragate and review the work of other reviewers and give a final suggestion on the content."
    """)
    .RegisterMessageConnector()
    .RegisterPrintMessage();

Define nest chat middleware

In [11]:
class NestChatMiddleware : IMiddleware
{
    private readonly IAgent seo;
    private readonly IAgent legal;
    private readonly IAgent ethics;
    private readonly IAgent meta;
    private readonly IAgent writer;

    public NestChatMiddleware(IAgent writer, IAgent seo, IAgent legal, IAgent ethics, IAgent meta)
    {
        this.writer = writer;
        this.seo = seo;
        this.legal = legal;
        this.ethics = ethics;
        this.meta = meta;
    }

    public string? Name => nameof(NestChatMiddleware);

    public async Task<IMessage> InvokeAsync(MiddlewareContext context, IAgent critic, CancellationToken cancellationToken = default)
    {
        // trigger only when the last message is from writer
        if (context.Messages.Last().From != writer.Name)
        {
            return await critic.GenerateReplyAsync(context.Messages, context.Options, cancellationToken);
        }

        var messageToReview = context.Messages.Last();
        var reviewPrompt = $"""
            Review the following content.

            {messageToReview.GetContent()}
            """;
        // SEO Review
        var seoReview = await critic.SendAsync(
            receiver: seo,
            message: reviewPrompt,
            maxRound: 1);
        var legalReview = await critic.SendAsync(
            receiver: legal,
            message: reviewPrompt,
            maxRound: 1);

        var ethicsReview = await critic.SendAsync(
            receiver: ethics,
            message: reviewPrompt,
            maxRound: 1);

        var reviews = seoReview.Concat(legalReview).Concat(ethicsReview);

        var metaReview = await critic.SendAsync(
            receiver: meta,
            message: "Aggregrate feedback from all reviewers and give final suggestions on the writing.",
            chatHistory: reviews,
            maxRound: 1);

        var lastReview = metaReview.Last();
        lastReview.From = critic.Name;

        return lastReview;
    }
}

In [12]:
// Orchestrate the nested chats to solve the task
var middleware = new NestChatMiddleware(
    writer: writer,
    seo: SEOReviewer,
    legal: LegalReviewer,
    ethics: EthicsReviewer,
    meta: MetaReviewer);

In [13]:
var nestChatCritic = critic
    .RegisterMiddleware(middleware)
    .RegisterPrintMessage();

In [14]:
conversation = await nestChatCritic.SendAsync(
    message: task,
    receiver: writer,
    maxRound: 3);

var finalResult = conversation.Last();

from: Writer
Title: "Unleashing the Power of Deep Learning with DeepLearning.AI"

Embark on a transformative journey with DeepLearning.AI, a pioneering platform revolutionizing the world of artificial intelligence. Offering cutting-edge courses and resources crafted by industry experts, DeepLearning.AI equips learners with the skills to delve into the depths of deep learning. From neural networks to computer vision, each module provides a comprehensive understanding of this rapidly evolving field. Whether you're a novice or a seasoned professional, DeepLearning.AI empowers you to unlock the full potential of AI technology. Join the community today and dive into the realm of deep learning with confidence and expertise.

from: SEO_Reviewer
As an SEO reviewer, my feedback on the content is:

- Incorporate relevant keywords like "deep learning courses," "AI training," and "online AI courses" to improve search engine visibility.
- Include a clear call-to-action at the end, encouraging reade